In [2]:
import numpy as np
def binomial(spot=100,strike=120,sigma=0.3,r=0.03,t=1,steps=1000):
    #Instead of actually making a tree using node structure, using an array of arrays
    #First array will have 1 element, second will have 2, third will have 3 and so on.
    #Store such a tree for the stock price as well as the option value.
    u=np.exp(sigma*np.sqrt(t/steps))
    d=np.exp(-sigma*np.sqrt(t/steps))
    p=(np.exp(r*(t/steps))-d)/(u-d)
    stock_tree=[]
    option_tree=[]
    #stock price tree construction
    #add the spot value of the stock
    stock_tree.append([spot])
    #calculating the rest of the sport values.
    #1000 steps means 1001 days.
    for i in range(1,steps+1):
        to_append=[]
        for j in range(len(stock_tree[i-1])):
            to_append.append(stock_tree[i-1][j]*u)
        to_append.append(stock_tree[i-1][-1]*d)
        stock_tree.append(to_append)
    #option value tree construction, but this is a bottom up approach, so we start from the future.
    #initially just make all elements 0.
    for i in range(steps+1):
        to_append=[]
        for j in range(i+1):
            to_append.append(0)
        option_tree.append(to_append)
    #first modify the option value at the last day
    for i in range(len(option_tree[-1])):
        #It is a call option
        option_tree[-1][i]=max(stock_tree[-1][i]-strike,0)
    #Now we use these to calulate rest of the option values
    c=np.exp(-r*(t/steps))
    for i in range(steps):
        for j in range(len(option_tree[steps-(i+1)])):
             option_tree[steps-(i+1)][j]=c*((p*option_tree[steps-i][j])+((1-p)*option_tree[steps-i][j+1]))
    return(option_tree[0][0])

In [21]:
price=[[],[],[]]
#Select a shock level
shock=0.1
for i in range(50, 160, 10):
    # we calculate option price for stock price of 50,60,...150
    price[0].append(binomial(spot=i))
    #print(price[0][-1])
    # we calculate option price for stock price of 50+shock,60+shock,...150+shock, helps for delta
    price[1].append(binomial(spot=i+shock))
    #print(price[1][-1])
    # we calculate option price for stock price of 50+shock*2,60+shock*2,...150+shock*2, helps for gamma
    price[2].append(binomial(spot=i+shock+shock))
    #print(price[2][-1])

In [24]:
#Use the binomial model you built in the previous assignment, to calculate various Greeks (Delta, Vega, Gamma, and Theta)
#For Delta and Gamma use Stock price in the range of 50 to 150 at $10 increments
# every time change in stock price is 0.001
delta=[[],[]]
for i in range(len(price[0])):
    #find delta at 50,60,...,150 
    delta[0].append((price[1][i]-price[0][i])/shock)
    print(delta[0][-1])
    
    #find delta at 50+shock,60+shock,...,150+shock
    delta[1].append((price[2][i]-price[1][i])/shock)
    #print(delta[1][-1])

print("_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_")

gamma=[]
for i in range(len(delta[0])):
    gamma.append((delta[1][i]-delta[0][i])/shock)
    print(gamma[-1])

0.003541333969410594
0.01972998763277653
0.06022935001842533
0.13352992863717938
0.2326139209922351
0.36288831354718276
0.4862296368785479
0.6109065409726
0.7035524040569285
0.784538841944169
0.8358021166514362
_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
0.0048280111079865295
-2.6367796834847468e-14
-7.605027718682322e-13
1.7541523789077473e-12
-2.4868995751603507e-12
1.1191048088221578e-11
-5.5067062021407764e-12
-1.1191048088221578e-11
4.263256414560601e-12
-5.2224891078367364e-11
6.892264536872972e-11


In [25]:
#first calculate the option price for the change in volatility
price=[[],[]]
#adjst shock level
shock=0.01
for i in range(5, 55, 5):
    #price at volatility 5%,10%,...,50%
    price[0].append(binomial(sigma=i/100))
    #print(price[0][-1])
    
    #price at volatility 5%+shock,10%+shock,...,50%+shock
    price[1].append(binomial(sigma=(i+shock)/100))
    #print(price[1][-1])

0.0017199418835851632
0.0017622341290281244
0.2991520320509176
0.3005498750885952
1.3066691417053033
1.3091772076822594
2.7674252068670677
2.77061492296685
4.464940095517142
4.468493836959519
6.292242602076462
6.296001188003746
8.19335778023337
8.197181374289313
10.139949119031057
10.143841038726796
12.116005240570207
12.119960310169619
14.101560853727687
14.105569101295163


In [26]:
#For Vega: change Vol from 5% to 50% at 5% increments
vega=[]
for i in range(len(price[0])):
    vega.append((price[1][i]-price[0][i])/shock)
    print(vega[-1])

0.004229224544296129
0.1397843037677593
0.25080659769560754
0.3189716099782469
0.35537414423769675
0.3758585927283953
0.38235940559427917
0.3891919695739432
0.3955069599411942
0.40082475674765305


In [28]:
#first calculate the option price for the change in time to maturity, while keeping the number of steps 1000.
price=[[],[]]
#adjust shock level
shock=0.001
for i in range(1, 13):
    #price for time period of 1 month, 2 months,...,12 months
    price[0].append(binomial(t=i/12))
    #print(price[0][-1])
    
    #price for time period of 1+1/1000th month, 2+1/1000th month,...,12 + 1/10th month
    price[1].append(binomial(t=(i+shock)/12))
    #print(price[1][-1])

0.06525120289175952
0.06546849754261476
0.44078768080521985
0.44127589000705997
0.9833535294501717
0.9839239523598158
1.5849771967206894
1.5855960024219702
2.2010788676351094
2.2017107941736604
2.8195901047264806
2.820206344814636
3.4284854927259785
3.429091208766755
4.025080878655411
4.025679021278128
4.608553442151794
4.609123037685039
5.184207520209151
5.184774338855035
5.739820712270716
5.740365605054583
6.292242602076462
6.2927867600274645


In [30]:
#For Theta: Change time to maturity in monthly decrements.
theta=[]
for i in range(len(price[0])):
    theta.append((price[1][i]-price[0][i])/shock)
    print(theta[-1])

0.21729465085523758
0.4882092018401174
0.5704229096440416
0.6188057012808112
0.631926538551042
0.6162400881555286
0.6057160407766382
0.5981426227172904
0.5695955332454261
0.5668186458835933
0.5448927838669348
0.5441579510021555
